In [18]:
# importing libraries
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier

# defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'webster-data445-bucket'
bucket = s3.Bucket(bucket_name)

# defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading the csv file
heart = pd.read_csv(file_content_stream)
# dropping missing values
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [19]:
# defining the input and target variables
X = heart[['male', 'age', 'currentSmoker', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

# splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [ ]:
# transforming to 0-1 scale
scaler = MinMaxScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test)

,male,age,currentSmoker,totChol,sysBP,BMI,heartRate,glucose
3785,0,38,0,214.0,115.0,25.69,80.0,65.0
1620,0,56,0,338.0,190.0,26.10,75.0,83.0
3643,0,36,0,188.0,112.0,22.54,63.0,73.0
371,1,55,0,239.0,159.0,32.35,75.0,71.0
2211,1,41,0,230.0,150.0,28.54,75.0,65.0


In [28]:
# lasso
lasso_cv = LassoCV(cv = 5).fit(X_train, Y_train)
lasso_cv.alpha_

# building lasso regression
lasso_md = Lasso(alpha = lasso_cv.alpha_).fit(X_train, Y_train)
lasso_md.coef_

array([ 0.06505177,  0.00732774,  0.0360036 ,  0.00024436,  0.00207421,
        0.00032419, -0.0002046 ,  0.00119299])

In [31]:
X_train_lasso = X_train[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]
X_test_lasso = X_test[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]

# logistic regression
logit_md = LogisticRegression().fit(X_train_lasso, Y_train)

# predicting on test
pred1 = logit_md.predict_proba(X_test_lasso)[:, 1]
pred1 = np.where(pred1 < 0.1, 0, 1)

# computing recall
recall_score(Y_test, pred1)

0.8660714285714286

In [36]:
# random forest classifier
RF = RandomForestClassifier(n_estimators = 500).fit(X_train, Y_train)

# extracting importance
RF.feature_importances_

array([0.02358636, 0.15263713, 0.02229293, 0.16792765, 0.18018311,
       0.17106712, 0.12401478, 0.15829093])

In [41]:
X_train_RF = X_train[['age', 'totChol', 'currentSmoker', 'sysBP', 'glucose']]
X_test_RF = X_test[['age', 'totChol', 'currentSmoker', 'sysBP', 'glucose']]

# building the random forest
RF = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train_RF, Y_train)

# predicting on test
pred2 = RF.predict_proba(X_test_RF)[:, 1]
pred2 = np.where(pred2 < 0.1, 0, 1)

# computing recall
recall_score(Y_test, pred2)

0.8303571428571429

In [42]:
print('The recall score of the Logistic Regression is', recall_score(Y_test, pred1))
print('The recall score of the Random Forest is', recall_score(Y_test, pred2))

# We would select the logistic regression because the recall score is higher.

The recall score of the Logistic Regression is 0.8660714285714286
The recall score of the Random Forest is 0.8303571428571429
